In [1]:
from functions_Chebyshev_basis import g_D_symbolic_coefficients_dict
from functions_Chebyshev_basis import restore_matrices
from functions_Chebyshev_basis import Augmented_Lagrangian
from functions_Chebyshev_basis import update_Lagrangian_coefficients
from functions_Chebyshev_basis import generate_M_d
import numpy as np
from scipy.integrate import quad
from functools import partial
import jax
import jax.numpy as jnp
from scipy.optimize import minimize
import sympy as sp

In [2]:
D = 6
d = 8
L = 10
rho = 10

In [3]:
expanded_result, coefficients_dict,polynomial,x = g_D_symbolic_coefficients_dict(D)
print("Expanded Result:")
print(expanded_result)
print("\nCoefficients Dictionary:")
for vars_tuple, coeff in coefficients_dict.items():
    print(f"Coefficient of {vars_tuple}: {coeff}")

orders_list = list(coefficients_dict.keys())
coefficients_list = list(coefficients_dict.values())
print(orders_list)
print(coefficients_list)

Expanded Result:
-x1**8*x2**8*x3**8*x4**8*x5**8*x6**8 + 0.166666666666667*x1**2 + 0.166666666666667*x2**2 + 0.166666666666667*x3**2 + 0.166666666666667*x4**2 + 0.166666666666667*x5**2 + 0.166666666666667*x6**2

Coefficients Dictionary:
Coefficient of (8, 8, 8, 8, 8, 8): -1.0
Coefficient of (2, 0, 0, 0, 0, 0): 0.16666666666666666
Coefficient of (0, 2, 0, 0, 0, 0): 0.16666666666666666
Coefficient of (0, 0, 2, 0, 0, 0): 0.16666666666666666
Coefficient of (0, 0, 0, 2, 0, 0): 0.16666666666666666
Coefficient of (0, 0, 0, 0, 2, 0): 0.16666666666666666
Coefficient of (0, 0, 0, 0, 0, 2): 0.16666666666666666
[(8, 8, 8, 8, 8, 8), (2, 0, 0, 0, 0, 0), (0, 2, 0, 0, 0, 0), (0, 0, 2, 0, 0, 0), (0, 0, 0, 2, 0, 0), (0, 0, 0, 0, 2, 0), (0, 0, 0, 0, 0, 2)]
[-1.0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]


In [4]:
x = sp.Symbol('x')

def chebyshev_polynomial(n):
    return 1/2*sp.chebyshevt(n, x)

def definite_integrate_chebyshev(n, a, b):
    T_n = chebyshev_polynomial(n)
    definite_integral = sp.integrate(T_n, (x, a, b))
    return definite_integral

In [5]:
# This block is for the matrix initialization and flatten as x_input

x_mu_D_L_list = [[] for _ in range(D)]
x_R_L_list = [[] for _ in range(D)]

# 9/28 update, we first generate a list of different moments of measure, then in the augumented lagrangian, we can generate the M_d matrix from our list of moments of measure
# This can fix the problem that when we try to differentiate the ϕn(µ), it actully differentiate the list of moments of measure not the M_d matrix 

# Define the mu_list
list_size = 2*d+3
my_list = np.array([definite_integrate_chebyshev(i,-1, 1) for i in range(2*d+3)])
del x
# # generate the list of list of both matrix
for l in range(L):
    for i in range(D):
        x_mu_D_L_list[i].append(my_list)
        x_R_L_list[i].append(np.random.uniform(-1, 1, size=(d+1, d+1)))

x_matrix_list_new = x_mu_D_L_list+x_R_L_list
x_input = np.array([])
for matrix_index in range(len(x_matrix_list_new)):
    for l in range(L):
        flattened_array = x_matrix_list_new[matrix_index][l].flatten()
        x_input = np.concatenate((x_input, flattened_array))
print(len(x_input))


6000


In [6]:
# Initialize the lagrangian coefficient as 0
Lagrangian_coefficient = 0
rho = 100

In [7]:
#implement the augumented lagrangian
aug_lagrangian_partial = partial(Augmented_Lagrangian, d=d, D=D, L=L, orders_list=orders_list,
                                 coefficients_list=coefficients_list,
                                 Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)

#use jax autogradient to generate the gradient
aug_lagrangian_partial_gradient = jax.grad(aug_lagrangian_partial)

In [8]:
print(aug_lagrangian_partial.keywords['Lagrangian_coefficient'])
print(aug_lagrangian_partial.keywords['rho'])

0
100


In [9]:
# iteration = 0
# while True:
#     iteration += 1
aug_lagrangian_partial = partial(Augmented_Lagrangian, d=d, D=D, L=L, orders_list=orders_list,
                                coefficients_list=coefficients_list,
                                Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)
aug_lagrangian_partial_gradient = jax.grad(aug_lagrangian_partial)

result = minimize(aug_lagrangian_partial, x0=x_input,
                method='L-BFGS-B',
                jac=aug_lagrangian_partial_gradient,
                options={
                    'gtol': 1e-3,             # Stopping criterion (relative gradient)
                    'ftol': 1e-4,             # Stopping criterion (absolute value)
                })

print("-"*40)
# print("This is {} iteration".format(iteration))

print("Minimum value of the Augmented Lagrangian function:", result.fun)
print("Was the optimization successful?", result.success)
print("Number of iterations:", result.nit)
print(result.message)

Lagrangian_coefficient = update_Lagrangian_coefficients(d,D,L,result.x,Lagrangian_coefficient,rho)
print("-"*40)

x_input = result.x
print("Lagrangian_coefficient:",Lagrangian_coefficient)
print("rho:",rho)

x_mu_D_L_list,_= restore_matrices(x_input,d,D,L)
print(x_mu_D_L_list)


----------------------------------------
Minimum value of the Augmented Lagrangian function: 0.008793934248387814
Was the optimization successful? True
Number of iterations: 265
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
----------------------------------------
Lagrangian_coefficient: -2.5861171469344373
rho: 100
[[array([ 1.46188413e-02,  1.40692171e-03, -5.36870112e-04, -1.82459491e-04,
        3.03131572e-03, -5.08224088e-05,  6.67588110e-03, -3.00134100e-04,
        1.59528979e-02, -2.37024536e-04,  3.82344154e-02,  4.17644075e-05,
        9.09549696e-02,  4.76012812e-04,  2.19415724e-01, -6.52844555e-05,
        5.31284019e-01,  0.00000000e+00, -3.09597524e-03]), array([ 1.66605475e-01,  3.46823092e-03, -1.13477054e-03,  3.40946983e-04,
        9.71713846e-04,  5.75976914e-04,  2.02033710e-03,  7.43423930e-04,
        5.02867457e-03,  1.70020639e-04,  1.26815900e-02, -4.41355390e-03,
        3.64545864e-02, -2.17917849e-02,  1.17263460e-01, -6.80126495e-02,
        4.27882132

In [10]:
result

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 0.008793934248387814
        x: [ 1.462e-02  1.407e-03 ...  2.709e-01  1.263e-01]
      nit: 265
      jac: [-1.091e-03 -1.556e-02 ... -8.636e-04 -2.507e-03]
     nfev: 272
     njev: 272
 hess_inv: <6000x6000 LbfgsInvHessProduct with dtype=float64>

In [187]:
x_input = x_input - 0.1*aug_lagrangian_partial_gradient(x_input)
print(x_input)

[ 0.05387476 -0.00633094 -0.01774413 ...  0.27339664 -0.23948748
 -0.31913447]


In [11]:
x_mu_D_L_list,_= restore_matrices(x_input,d,D,L)
print(x_mu_D_L_list)

[[array([ 1.46188413e-02,  1.40692171e-03, -5.36870112e-04, -1.82459491e-04,
        3.03131572e-03, -5.08224088e-05,  6.67588110e-03, -3.00134100e-04,
        1.59528979e-02, -2.37024536e-04,  3.82344154e-02,  4.17644075e-05,
        9.09549696e-02,  4.76012812e-04,  2.19415724e-01, -6.52844555e-05,
        5.31284019e-01,  0.00000000e+00, -3.09597524e-03]), array([ 1.66605475e-01,  3.46823092e-03, -1.13477054e-03,  3.40946983e-04,
        9.71713846e-04,  5.75976914e-04,  2.02033710e-03,  7.43423930e-04,
        5.02867457e-03,  1.70020639e-04,  1.26815900e-02, -4.41355390e-03,
        3.64545864e-02, -2.17917849e-02,  1.17263460e-01, -6.80126495e-02,
        4.27882132e-01,  0.00000000e+00, -3.09597524e-03]), array([ 9.96403709e-02,  3.97270225e-03, -6.80138184e-04, -3.81357483e-04,
        1.56760460e-03, -8.99414410e-04,  3.68947535e-03, -1.84541273e-03,
        9.28049868e-03, -3.49901806e-03,  2.34750804e-02, -5.34636492e-03,
        6.15424922e-02, -4.72445360e-03,  1.69083408e

In [189]:
aug_lagrangian_partial_gradient(x_input)

Array([-0.34379572, -2.3939903 , -5.681319  , ...,  1.783972  ,
        0.8618355 ,  0.75519305], dtype=float32)

In [12]:
x_M_D_L_list = generate_M_d(x_mu_D_L_list,d,D,L)
l_product_list = []
for l in range(L):
    moment_product = 1
    for i in range(D):
        moment_product *= x_M_D_L_list[i][l][0,0]
    l_product_list.append(moment_product)

max_index = l_product_list.index(max(l_product_list))
x_output = [x_mu_D_L_list[i][max_index][1] for i in range(D)]
print(x_output)

[0.0034682309226637268, 0.009145529286534813, 0.013348093252105717, -0.0046068726704518136, 0.007370292368391903, -0.017143204247443224]


In [14]:
# Define the function based on the given equation
def compute_function(x):
    D = len(x)
    sum_term = (1 / D) * np.sum(2 * x**2 - 1)
    product_term = np.prod(np.cos(8 * np.arccos(x)))
    result = sum_term - product_term
    return result

x = np.array(x_output)
result_out = compute_function(x)
print(result_out)
print((result_out+2)/2)


-1.9793666465112825
0.010316676744358766
